## Zadanie domowe: Projektowanie filtru metodą okien

Zauważone w zadaniu z filtracją artefakty są efektem zastosowania filtru idealnego (wycięcia odpowiednich częstotliwości).
Aby je usunąć, należy filtr idealny w dziedzinie przestrzennej przemnożyć przez dwuwymiarowe okno (stworzone na podstawie jednowymiarowego).

1. Zdefiniuj rozmiar filtra (np. 21).

2. Stwórz okno np. Hanninga (`np.hanning`) o zadanym rozmiarze.

3. Na podstawie tego okna stwórz okno dwuwymiarowe: $F = f^Tf$.
Można to zrealizować prostą instrukcją:

        hanning2d = np.outer(hanning, hanning)

4. Stwórz wzorcowy filtr idealny tak samo jak w poprzednim ćwiczeniu.
Jego rozmiar musi być taki sam jak zdefiniowany w punkcie pierwszym.

5. Generalna zasada jest taka, że należy obliczyć odwrotną transformatę Fouriera filtru idealnego.
Aby zagwarantować jednak dobrą lokalizację i symetryczność tworzonego filtru wykorzystuje się dodatkowe rotacje i przesunięcia.

        FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
        FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
        FilterFidft = cv2.idft(np.float32(FilterFRot3),
                        flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
        FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)

6. Aby otrzymać poprawny filtr należy teraz pomnożyć dwuwymiarowe okno ze stworzonym filtrem idealnym w dziedzinie przestrzennej.

7. Stworzony filtr powinien zostać znowu przeniesiony do dziedziny częstotliwości.
Należy jednak rozszerzyć go tak, aby miał taki sam rozmiar jak filtrowany obraz.
Operację tę wykonaj tak samo jak w poprzednim ćwiczeniu (`cv2.copyMakeBorder`).
Oblicz transformatę Fouriera i wykonaj przesunięcie.
Następnie oblicz wartość bezwzględną (dzięki temu filtr nie będzie wpływał na fazę).
Wynikiem jest poprawiony filtr w dziedzinie częstotliwości.

8. Przeprowadź filtrację obrazu *lena.bmp* za pomocą zaprojektowanego filtra.
Wyświetl filtr, oraz wynik filtracji.
Do wyświetlenia filtru możesz użyć przestrzeń częstotliwości wygenerowaną w poprzednim zadaniu (*FSpaceRowsM* i *FSpaceColsM*).

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

# Load required files
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate

lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)

def fourier_log(image):
    image = image.astype("float32")
    F_dft = cv2.dft(image,flags=cv2.DFT_COMPLEX_OUTPUT)
    F_dft_shift = np.fft.fftshift(F_dft, [0,1])
    amp, phase = cv2.cartToPolar(F_dft_shift[:,:,0], F_dft_shift[:,:,1])
    Alog = np.log10(amp+1)

    return F_dft_shift,Alog,phase


def fourier_reverse(image):
    image_shift = np.fft.ifftshift(image, [0,1])
    ifft = cv2.idft(image_shift, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    img_ifft = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
    finall_image= np.round(img_ifft).astype('uint8')
    return finall_image


In [ ]:
def ideal_filter(image, size,f):
    size = 21
    han_win = np.hanning(size)
    hanning2d = np.outer(han_win,han_win)
    X,Y = np.shape(lena)
    window_size=[size,size]
    FSpaceRows = 2*np.fft.fftshift(np.fft.fftfreq(window_size[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, window_size[1]]))
    FSpaceCols = 2*np.fft.fftshift(np.fft.fftfreq(window_size[1]))
    FSpaceColsM = np.outer(np.ones([1, window_size[0]]), FSpaceCols)    
    FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

    Filter_gorno = FreqR >= f
    FilterF = Filter_gorno


    FilterFRot = np.rot90(np.fft.fftshift(np.rot90(FilterF, 2)), 2)
    FilterFRot3 = np.dstack((FilterFRot, np.zeros(FilterFRot.shape)))
    FilterFidft = cv2.idft(np.float32(FilterFRot3),flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)

    FilterFI = np.rot90(np.fft.fftshift(FilterFidft[:, :, 0]), 2)


    Filtr_all=hanning2d*FilterFI

    Filtr_bor = cv2.copyMakeBorder(Filtr_all,X-size,0,Y-size,0,cv2.BORDER_CONSTANT,value=0)

   
    Filtr_bor = Filtr_bor.astype("float32")
    F_fcn_filtr = cv2.dft(Filtr_bor,flags=cv2.DFT_COMPLEX_OUTPUT)
    F_fcn_filtr = np.fft.fftshift(F_fcn_filtr, [0,1])

    F_fcn_filtr_abs = np.abs(F_fcn_filtr)



    size2 = image.shape
    FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(size2[0]))
    FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, size2[1]]))
    FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(size2[1]))
    FSpaceColsM = np.outer(np.ones([1, size2[0]]), FSpaceCols)

    figFilter = plt.figure()
    axsFilter = figFilter.add_subplot(projection='3d')
    axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, F_fcn_filtr_abs[:,:,0], rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
    
    image_dft = fourier_log(image)[0]
    image_filtr =image_dft*F_fcn_filtr_abs

    image_rev = fourier_reverse(image_filtr)
    diff = np.abs(lena-image_rev)


    f, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(20,6))
    ax1.imshow(lena, 'gray')
    ax1.axis('off')
    ax1.set_title('Oryginał')
    ax2.imshow(image_rev, 'gray')
    ax2.axis('off')
    ax2.set_title('Obraz po przekształceniach')
    ax3.imshow(diff, 'gray')
    ax3.axis('off')
    ax3.set_title('Różnica')
   
ideal_filter(lena,21,0.2)